In [1]:
import pandas as pd
import numpy as np
import psycopg2

In [2]:
def luo_yhteys():
    try:
        connection = psycopg2.connect(
            host="192.168.0.15",
            database="VehicleRegisterDatabase",
            user="postgres",
            password="1337"
        )
        #print("Connection to PostgreSQL database successful")
        print(connection)
    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL database:", error)

    try:
        cursor = connection.cursor()
        print("Cursor created successfully")
    except (Exception, psycopg2.Error) as error:
        print("Error while creating a cursor:", error)

    return cursor


In [14]:
cursor = luo_yhteys()
query = f'''
    SELECT kunta, ajoneuvoluokka, ensirekisterointipvm
    FROM ajoneuvorekisteroinnit
    WHERE kunta = '179' AND ajoneuvoluokka = 'M1' AND ensirekisterointipvm >= '2010-01-01' AND ensirekisterointipvm <= '2022-01-01'
    ORDER BY ensirekisterointipvm
'''
try:
    cursor.execute(query)
    # Fetch and print the results
    records = cursor.fetchall()
    df = pd.DataFrame(records, columns=["kunta", "ajoneuvoluokka", "ensirekisterointopvm"])
    print(df)

    # Commit the transaction
    #connection.commit()
except (Exception, psycopg2.Error) as error:
    print("Error while executing SQL query:", error)


<connection object at 0x0000018B093059C0; dsn: 'user=postgres password=xxx dbname=VehicleRegisterDatabase host=192.168.0.15', closed: 0>
Cursor created successfully
      kunta ajoneuvoluokka ensirekisterointopvm
0       179             M1           2010-01-02
1       179             M1           2010-01-02
2       179             M1           2010-01-02
3       179             M1           2010-01-04
4       179             M1           2010-01-04
...     ...            ...                  ...
20276   179             M1           2020-05-07
20277   179             M1           2020-08-20
20278   179             M1           2021-05-18
20279   179             M1           2021-07-23
20280   179             M1           2021-11-01

[20281 rows x 3 columns]


In [9]:
def ensirekisteroinnit(alku, loppu, merkki, kunta):
    cursor = luo_yhteys()
    query = f'''
        SELECT kunta, ajoneuvoluokka, ensirekisterointipvm
        FROM ajoneuvorekisteroinnit
        WHERE kunta = '179' AND ajoneuvoluokka = 'M1' AND ensirekisterointipvm >= '2020-01-01' AND ensirekisterointipvm <= '2022-01-01'
        ORDER BY ensirekisterointipvm
    '''
    try:
        cursor.execute(query)
        # Fetch and print the results
        records = cursor.fetchall()
        df = pd.DataFrame(records, columns=["kunta", "ajoneuvoluokka", "ensirekisterointopvm"])
        print(df)
        
        # Commit the transaction
        #connection.commit()
    except (Exception, psycopg2.Error) as error:
        print("Error while executing SQL query:", error)


In [ ]:
def ensirekisteroinnit(vuosi merkki):
    cursor = luo_yhteys()
    query = f'''
        SELECT ensirekisterointipvm, merkki
        FROM ajoneuvorekisteroinnit
        WHERE merkki =  AND ajoneuvoluokka = 'M1' AND ensirekisterointipvm >= '2020-01-01' AND ensirekisterointipvm <= '2022-01-01'
        ORDER BY ensirekisterointipvm
    '''
    try:
        cursor.execute(query)
        # Fetch and print the results
        records = cursor.fetchall()
        df = pd.DataFrame(records, columns=["kunta", "ajoneuvoluokka", "ensirekisterointopvm"])
        print(df)
        
        # Commit the transaction
        #connection.commit()
    except (Exception, psycopg2.Error) as error:
        print("Error while executing SQL query:", error)
